# draw trade

画出两个币的走势关系


In [18]:
pair1 = "BTC/USDT"
pair2 = "WLD/USDT"

In [19]:
import polars as pl
import ccxt

from datetime import datetime, timedelta


exchange_ins = ccxt.binance()

def fetch_ohlcv(symbol, timeframe="5m", since=None, limit=500):

    ohlcv = exchange_ins.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)

    df = pl.DataFrame(
        ohlcv,
        schema={
            "timestamp": pl.UInt64,
            "open": pl.Float64,
            "high": pl.Float64,
            "low": pl.Float64,
            "close": pl.Float64,
            "volume": pl.Float64,
        },
        orient="row",
    )

    return df.with_columns((pl.col("timestamp")).cast(pl.Datetime("ms")).alias("date"))

In [21]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


def show_indicator(df1, df2):
    # 创建子图
    fig = make_subplots(
        rows=1, cols=2, shared_xaxes=True, subplot_titles=(pair1, pair2)
    )

    # 添加第一个数据框的线图
    fig.add_trace(
        go.Candlestick(
            x=df1["date"],
            open=df1["open"],
            high=df1["high"],
            low=df1["low"],
            close=df1["close"],
            name=pair1,
        ),
        row=1,
        col=1,
    )

    # 添加第二个数据框的线图
    fig.add_trace(
        go.Candlestick(
            x=df2["date"],
            open=df2["open"],
            high=df2["high"],
            low=df2["low"],
            close=df2["close"],
            name=pair2,
        ),
        row=1,
        col=2,
    )

    # 更新布局
    fig.update_layout(
        title_text="Price over Time Comparison",
        height=600,  # 设置图表高度
        width=1000,  # 设置图表宽度
    )
    fig.show()

## 1m


In [ ]:
one_day = int((datetime.now() - timedelta(days=1)).timestamp()) * 1000
df1 = fetch_ohlcv(pair1, timeframe="1m", since=one_day, limit=60 * 24)
df2 = fetch_ohlcv(pair2, timeframe="1m", since=one_day, limit=60 * 24)
show_indicator(df1, df2)

## 5m


In [ ]:
df1 = fetch_ohlcv(pair1)
df2 = fetch_ohlcv(pair2)
show_indicator(df1, df2)

## 15 m


In [ ]:
limit = 10
three_day = int((datetime.now() - timedelta(days=limit)).timestamp()) * 1000
df1 = fetch_ohlcv(pair1, timeframe="15m", since=three_day, limit=4 * 24 * limit)
df2 = fetch_ohlcv(pair2, timeframe="15m", since=three_day, limit=4 * 24 * limit)
show_indicator(df1, df2)

## 1 day


In [ ]:
limit = 23
one_day = int((datetime.now() - timedelta(days=limit)).timestamp()) * 1000
df1 = fetch_ohlcv(pair1, timeframe="1d", since=one_day, limit=limit)
df2 = fetch_ohlcv(pair2, timeframe="1d", since=one_day, limit=limit)
show_indicator(df1, df2)